In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

import re
import os
import string
import operator
from itertools import islice
from collections import Counter

from datetime import date
import matplotlib.dates as dates

from nltk import ngrams
import nltk.probability

In [4]:
sns.set_style('darkgrid')
sns.set_context('paper')
pd.set_option('display.max_colwidth', None)

In [5]:
punct_signs = list(string.punctuation)
punct_signs.append('…')
punct_signs.append('¿')
punct_signs.append('•')
punct_signs.append('”')
punct_signs.append('“')
punct_signs.append('–')
punct_signs.append('∑')
punct_signs.remove('&')


## General Information 

In [7]:
df = pd.read_csv('data/courses.csv')
df.columns

Index(['ID', 'Country', 'Univeristy', 'G2RWorldRank', 'G2RNationalRank',
       'QSWorldUniversityRanking2021', 'Program',
       'SchoolOfferingCourseProgram', 'CourseDescription', 'Course Unit',
       'CourseUnitGoals', 'CourseUnitOutcomes'],
      dtype='object')

In [10]:

print('Total of courses: ', len(df))

Total of courses:  503


## Number of courses by country

In [25]:
df_ = df.groupby(['Country']).agg({'ID':'count'})

df_.columns = ['Number of courses']
df_ = df_.sort_values('Number of courses', ascending= False)
df_.loc['Total', 'Number of courses'] = df_['Number of courses'].sum()
df_.astype(int)

,Number of courses
Country,
USA,361
Canada,32
Switzerland,24
UK,20
Singapure,19
Hong Kong,12
Germany,10
Sweden,7
Belgium,5


## Total of countries

In [26]:
len(df_)

17

## Number of courses by University (Top 20)

In [30]:
df_ = df.groupby(['Univeristy','G2RWorldRank', 'G2RNationalRank',
       'QSWorldUniversityRanking2021' ]).agg({'ID':'count'})

df_.columns = ['Number of courses']
df_ = df_.sort_values('Number of courses', ascending= False)

df_= df_.astype(int)
df_.head(20)

,,,,Number of courses
Univeristy,G2RWorldRank,G2RNationalRank,QSWorldUniversityRanking2021,
Carnegie Mellon University,1,1,51.0,42
University of Texas at Austin,15,14,71.0,38
University of Washington,11,10,72.0,24
"University of California, Irvine",21,18,210.0,21
MIT,2,2,1.0,20
University of Waterloo,22,2,166.0,20
University of Pennsylvania,24,19,16.0,20
University of Southern California,10,9,121.0,17
Cornell University,19,16,18.0,15


## Total of Universities

In [28]:
len(df_)

66

## Number of courses by program (top 20)

In [37]:
df_ = df.groupby(['Program' ]).agg({'ID':'count'})

df_.columns = ['Number of courses']
df_ = df_.sort_values('Number of courses', ascending= False)

df_= df_.astype(int)
df_.head(20)

,Number of courses
Program,
Engineering Ethics,4
Artificial Intelligence,4
Ethics and Policy Issues in Computing,3
Introduction to Software Engineering,3
Computer Science,3
Social Implications of Computer Technology,2
"Computers, Ethics, and Public Policy",2
Computers and Society,2
Senior Design Project,2


## Total of Programs

In [38]:
len(df_)

475

## Total of courses by SchoolOfferingCourseProgram (top 20)

In [39]:
df_ = df.groupby(['SchoolOfferingCourseProgram' ]).agg({'ID':'count'})

df_.columns = ['Number of courses']
df_ = df_.sort_values('Number of courses', ascending= False)

df_= df_.astype(int)
df_.head(20)

,Number of courses
SchoolOfferingCourseProgram,
Engineering School,18
College of Engineering,18
COLLEGE OF ENGINEERING,17
Whiting School of Engineering,12
School of Electrical Engineering,12
Informatics,12
School of continuing and lifelong education,9
Computer Science Department,9
Department of Computer Science and Engineering,9


## Total of Schools Offering Program

In [40]:
len(df_)

52

##  Total of courses including Course Description

In [45]:
df['CourseDescription'].count()

478

##  Total of courses including Goals. 

In [46]:
df['CourseUnitGoals'].count()

74

## Total of courses including outcomes.

In [47]:
df['CourseUnitOutcomes'].count()

76

## Total of courses including Description, Goals and Outcomes. 

In [53]:
df_ = df.assign(count=df.loc[:, ['CourseUnitOutcomes','CourseUnitGoals', 'CourseDescription']].notna().sum(axis=1))
df_ = df[df_['count'] == 3]
len(df_)

46